### CIFAR-10 Image Classification Using CNN

#### Import Libraries and Datasets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 19s 0us/step


In [10]:
# 50000 is the number of samples, 32 is the width, 32 is the length
# and 3 is due to 3 channels for RGB values
X_train.shape

(50000, 32, 32, 3)